In [81]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [82]:
df = pd.read_csv('fourCollections.csv', dtype={'collection_slug': 'str', 'asset_id': 'int', 'asset_name': 'str', 'owner_username': 'str', 'owner_address': 'str', 'event_type': 'str'})
df.drop('Unnamed: 0.1', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('collection_slug', axis=1,inplace=True)
df.drop('asset_contract_date', axis=1, inplace=True)
df.drop('asset_name', axis=1, inplace=True)
df.drop('asset_id', axis=1, inplace=True)
df.drop('owner_username', axis=1, inplace=True)
df.drop('owner_address', axis=1, inplace=True)
df['event_timestamp'] = pd.to_datetime(df['event_timestamp']).dt.strftime('%m/%d/%Y')
df.drop_duplicates(keep='first')
df['year'] = pd.to_datetime(df['event_timestamp']).dt.strftime('%Y')
df['month'] = pd.to_datetime(df['event_timestamp']).dt.strftime('%m')
df['count'] = 1
df

,collection_name,event_timestamp,event_type,year,month,count
0,Azuki,06/17/2022,created,2022,06,1
1,Azuki,06/17/2022,created,2022,06,1
2,Azuki,06/17/2022,created,2022,06,1
3,Azuki,06/17/2022,created,2022,06,1
4,Azuki,06/17/2022,created,2022,06,1
...,...,...,...,...,...,...
1465480,Crypto Coven,06/10/2022,offer_entered,2022,06,1
1465481,Crypto Coven,06/10/2022,offer_entered,2022,06,1
1465482,Crypto Coven,06/10/2022,offer_entered,2022,06,1
1465483,Crypto Coven,06/10/2022,offer_entered,2022,06,1


In [90]:
df.groupby('month', as_index=False).agg({"count": "sum"})

,month,count
0,01,235430
1,02,176984
2,03,131914
3,04,107738
4,05,288377
5,06,173491
6,07,61318
7,08,73618
8,09,32003
9,10,23650
